In [2]:
import ee
ee.Authenticate()
ee.Initialize(project='ee-aryr1103')

In [4]:
def dataset_value(dataset, lon, lat):
    point = ee.Geometry.Point([lon, lat])
    if isinstance(dataset, ee.imagecollection.ImageCollection):
        dataset = dataset.mean()
    value = dataset.reduceRegion(
        reducer=ee.Reducer.first(), 
        geometry=point, 
        scale=30
    ).getInfo()
    return value
startYear = '2020-07-01'
endYear = '2020-09-01'

In [5]:
elevation = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
temperature = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')
biomass = ee.ImageCollection("WCMC/biomass_carbon_density/v1_0").select('carbon_tonnes_per_ha')
humidity = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H").select('Qair_f_inst')
precipitation = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").select('precipitation')
fires = ee.FeatureCollection("USFS/GTAC/MTBS/burned_area_boundaries/v1")

In [6]:
blackest = [1, 0]

# Unix epoch time for January 1, 2020
epoch_2020 = 1577836800

for i in range(2, 69420):
    fire_feature = ee.Feature(fires.toList(i).get(i-1))
    
    date = fire_feature.get('Ig_Date').getInfo()
    ans = fire_feature.get('BurnBndAc').getInfo()

    # Check if the fire is after 2020
    if date > epoch_2020 and ans > blackest[1]:
        blackest = [i, ans]
        print(i, ans)


2 8385
4 19441


KeyboardInterrupt: 

In [ ]:
print(blackest)

[1, 0]


In [7]:
n=479
fire_feature = ee.Feature(fires.toList(n).get(n-1))
fire_bounds = fire_feature.geometry().bounds()
buffer_distance = 1

# Create a rectangle around the fire perimeter with a buffer
rectangle = fire_bounds.buffer(buffer_distance).bounds()

In [8]:
def generate_grid(corners, point_spacing):
    coords = []
    min_lon, min_lat = corners[0]
    max_lon, max_lat = corners[1]

    lon = min_lon
    while lon < max_lon:
        lat = min_lat
        while lat < max_lat:
            coords.append([lon, lat])
            lat += point_spacing
        lon += point_spacing
    return coords


In [20]:
# Define the spacing between points (in degrees)
point_spacing = 0.01  # Example: approximately 111 meters at the equator

# Extract the lower-left and upper-right corners of the rectangle
rect_coords = rectangle.bounds().getInfo()['coordinates'][0]
lower_left = rect_coords[0]  # Lower-left corner
upper_right = rect_coords[2]  # Upper-right corner
    
grid_points = generate_grid([lower_left, upper_right], point_spacing)
print(lower_left, upper_right)
center = [0, 0]
center[0] = (lower_left[0]+upper_right[0])/2
center[1] = (lower_left[1]+upper_right[1])/2
print(center)

[-115.56751081403489, 42.39835008521034] [-114.88116823901456, 42.92725758477888]
[-115.22433952652472, 42.662803834994605]


In [32]:
import geemap.foliumap as geemap
import folium
Map = geemap.Map()
for point in grid_points:
    color = 'blue'
    if fire_feature.geometry().contains(ee.Geometry.Point(point)).getInfo():
        color = 'red'
    folium.Circle(
        location=point[::-1],
        radius=10,
        color=color
    ).add_to(Map)
Map

KeyboardInterrupt: 

In [33]:
data = []
for lon, lat in grid_points:
    elev = dataset_value(elevation, lon, lat)
    temp = dataset_value(temperature, lon, lat)
    bio = dataset_value(biomass, lon, lat)
    hum = dataset_value(humidity, lon, lat)
    precip = dataset_value(precipitation, lon, lat)
    burned = fire_feature.geometry().contains(ee.Geometry.Point([lon, lat])).getInfo()
    data.append([lon, lat, elev['elevation'], temp['LST_Day_1km'], bio['carbon_tonnes_per_ha'], hum['Qair_f_inst'], precip['precipitation'], burned])

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [34]:
print(data)

[[-115.56751081403489, 42.39835008521034, 1403, 15228.566251415628, 16.145305633544922, 0.004190677776932716, 0.03651343286037445, False], [-115.56751081403489, 42.40835008521034, 1362, 15176.279721848363, 16.145305633544922, 0.004190677776932716, 0.036475539207458496, False], [-115.56751081403489, 42.418350085210335, 1347, 15201.613549403557, 16.145305633544922, 0.004190677776932716, 0.036475539207458496, False], [-115.56751081403489, 42.42835008521033, 1346, 15197.311868970159, 16.145305633544922, 0.004190677776932716, 0.036475539207458496, False], [-115.56751081403489, 42.43835008521033, 1351, 15199.41027932961, 16.145305633544922, 0.004190677776932716, 0.036475539207458496, False], [-115.56751081403489, 42.44835008521033, 1358, 15203.276389826693, 16.145305633544922, 0.004190677776932716, 0.036475539207458496, False], [-115.56751081403489, 42.45835008521033, 1330, 15138.654323725055, 16.145305633544922, 0.004190677776932716, 0.036475539207458496, False], [-115.56751081403489, 42.46

In [35]:
import csv

# Open a new CSV file
with open('nishyorz.csv', 'w', newline='') as file:
    writer = csv.writer(file)

    # Write each row of the array to the CSV
    for row in data:
        writer.writerow(row)